# Gene Ontology (GO)

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://geneontology.org/docs/downloads/

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import itertools
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

This notebook was run on: 2020-06-28 
Python version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]


# Initialization

In [32]:
%%appyter code_eval

{% set attribute = ChoiceField(
    name='attribute',
    label='Attribute',
    choices={
        'Biological Process': "'Process'",
        'Cellular Component': "'Component'",
        'Molecular Function': "'Function'"
    },
    default='Cellular Component',
    section='data'
) %}

### Load Mapping Dictionaries

In [7]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:10<00:00,  3.50s/it]


### Output Path

In [33]:
%%appyter code_exec

output_name = 'go_' + {{attribute}}.lower()

path = 'Output/GO-' + {{attribute}}
if not os.path.exists(path):
    os.makedirs(path)

```python
output_name = 'go_' + 'Component'.lower()
path = 'Output/GO-' + 'Component'
if not os.path.exists(path):
    os.makedirs(path)
```

In [34]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Gene Ontology Resource'
) %}

# Load Data

In [49]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.gaf$',
    name='gaf', 
    label='Gene Assocation File (gaf)', 
    default='Input/GO/goa_human.gaf',
    section='data')
}}, skiprows=31, header=None, usecols=[2, 3, 4, 6, 8], sep='\t',
    names=['DB Object Symbol', 'Qualifier', 'GO ID', 'Evidence Code', 'Aspect'])

```python
df = pd.read_csv('Input/GO/goa_human.gaf', skiprows=31, header=None, usecols=[2, 3, 4, 6, 8], sep='\t',
    names=['DB Object Symbol', 'Qualifier', 'GO ID', 'Evidence Code', 'Aspect'])
```

In [50]:
df.head()

,DB Object Symbol,Qualifier,GO ID,Evidence Code,Aspect
0,NUDT4B,NaN,GO:0003723,IEA,F
1,NUDT4B,NaN,GO:0005829,IDA,C
2,NUDT4B,NaN,GO:0008486,IEA,F
3,NUDT4B,NaN,GO:0046872,IEA,F
4,NUDT4B,NaN,GO:0052840,IEA,F


In [51]:
df.shape

(502842, 5)

## Load Ontology

In [52]:
%%appyter code_exec

tree = ET.parse({{FileField(
    constraint='.*\.owl$',
    name='ontology', 
    label='Ontology (owl)', 
    default='Input/GO/go.owl',
    section='data')
}})
root = tree.getroot()

```python
tree = ET.parse('Input/GO/go.owl')
root = tree.getroot()
```

# Pre-process Data

## Get Relevant Data

In [53]:
%%appyter code_exec

# Get only desired attribute:
# P for Biological Process, C for Cellular Component, F for Molecular Function
df = df[df['Aspect'] == {{attribute}}[0]]
# Drop data inferred from electronic annotation
df = df[df['Evidence Code'] != 'IEA']
# Drop NOT in qualifier
df = df[df['Qualifier'] != 'NOT']

```python
# Get only desired attribute:
# P for Biological Process, C for Cellular Component, F for Molecular Function
df = df[df['Aspect'] == 'Component'[0]]
# Drop data inferred from electronic annotation
df = df[df['Evidence Code'] != 'IEA']
# Drop NOT in qualifier
df = df[df['Qualifier'] != 'NOT']
```

In [54]:
df.shape

(146472, 5)

In [55]:
df = df[['DB Object Symbol', 'GO ID']]
df.head()

,DB Object Symbol,GO ID
1,NUDT4B,GO:0005829
64,TRBV19,GO:0042105
71,IGKV2-28,GO:0005576
72,IGKV2-28,GO:0005576
73,IGKV2-28,GO:0005576


## Build GO Ontology Map

In [42]:
ns = {
    'owl': 'http://www.w3.org/2002/07/owl#',
    'obo': 'http://www.geneontology.org/formats/oboInOwl#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'
}

In [43]:
mapping = {'GO ID': [], 'Label': []}

for child in root.findall('owl:Class', ns):
    id = child.find('obo:id', ns)
    label = child.find('rdfs:label', ns)
    if id is not None and label is not None:
        mapping['GO ID'].append(id.text)
        mapping['Label'].append(label.text)

onto_meta = pd.DataFrame(mapping).set_index('GO ID')
onto_meta.head()

,Label
GO ID,
GO:0000001,mitochondrion inheritance
GO:0000002,mitochondrial genome maintenance
GO:0000003,reproduction
GO:0000005,obsolete ribosomal chaperone activity
GO:0000006,high-affinity zinc transmembrane transporter a...


## Map GO IDs

In [56]:
df['GO ID'] = onto_meta['Label'].reindex(df['GO ID']).reset_index(drop=True)
df = df.set_index('DB Object Symbol')
df.index.name = 'Gene Symbol'
df.columns = ['GO Term']
df.head()

,GO Term
Gene Symbol,
NUDT4B,alpha-beta T cell receptor complex
TRBV19,extracellular region
IGKV2-28,extracellular region
IGKV2-28,extracellular region
IGKV2-28,extracellular region


# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [57]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

100%|██████████| 146472/146472 [00:00<00:00, 648199.68it/s]


(36127, 1)

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)